# Logistic Regression for Classification of Digits

I get pretty good results on 0 vs 1, implying that telling the difference between a "0" and a "1" is not a hard problem anymore. Also pretty epic results on predicting 0 through 9.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

## Load "zeros" and "ones" training data

In [ ]:
digits = load_digits(2)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=9, figsize=(20,10))
plt.gray()

for i in range(9):
    axes[i].imshow(digits.images[i])
    axes[i].set_title(digits.target[i])

plt.show()

## Split our data into training and test sets

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(digits.images, digits.target)

train_x, test_x = [x.reshape(x.shape[0], -1).T / 16 for x in [train_x, test_x]]
train_y, test_y = [y.reshape(1, y.shape[0]) for y in [train_y, test_y]]

train_x.shape, train_y.shape, test_x.shape, test_y.shape

## Whip up a quick gradient descent loop

In [ ]:
def gradient_descent(X, y, step_size, epochs):
    w, b = np.random.randn(X.shape[0], 1), 0
    
    for i in range(epochs):
        yhat = 1 / (1 + np.exp( - (np.dot(w.T, X) + b)))

        if (i % (epochs/10) == 0):  print("Cost at iteration {}: {}".format(i, -np.mean(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))))

        w = w - step_size * (1 / w.shape[0]) * np.dot(X, (yhat - y).T) 
        b = b - step_size * np.mean(yhat - y)

    return w, b

## Let's see how accurate we got

In [ ]:
w, b = gradient_descent(train_x, train_y, 0.001, 100000)
        
predict = lambda X, w, b: np.array((1 / (1 + np.exp(-1 * (np.dot(w.T, X) + b)))) > 0.5, dtype=int)
accuracy = lambda y, yhat: np.mean(y == yhat) * 100

print("Accuracy on train set: {}%".format(accuracy(train_y, predict(train_x,w,b))))
print("Accuracy on test set: {}%".format(accuracy(test_y, predict(test_x,w,b))))

# Let's try predicting more than binary and try predict 0 through 9

In [ ]:
digits10 = load_digits(10)
fig, axes = plt.subplots(nrows=1, ncols=10, figsize=(20,10))
plt.gray()

for i in range(10):
    axes[i].imshow(digits10.images[i])
    axes[i].set_title(digits10.target[i])

plt.show()

# Logistic Regression for multiple classes

In [ ]:
# need to wrangle the data to vectors
scale = digits10.images.max()

# a matrix with each row representing the image, and "normalized" by dividing through max
X = digits10.images.reshape(digits10.images.shape[0], -1) / scale

# one hot encode the output
y = np.zeros((len(digits10.target), 10))
y[np.arange(len(digits10.target)), digits10.target] = 1

In [ ]:
# for convenience, change X and y to be a matrix where each *column* represents the training input/output
train_x, test_x, train_y, test_y = (np.array(set.T) for set in train_test_split(X, y))

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

In [ ]:
def dimensions(X,y): 
    '''
    c = the number of classifications possible for each given ouput
    D = the number of features
    m = number of training samples
    '''
    c = y.shape[0]
    D, m = X.shape
    return c, D, m

c, D, m = dimensions(train_x, train_y)
print(c,D,m)

$\mathbf{z} = \mathbf{w}^T\mathbf{X} + \mathbf{b}$

$ \mathbf{z} = 
\
\begin{bmatrix} \
z_1^{(1)} & z_1^{(2)} & \cdots & z_1^{(m)} \\
z_2^{(1)} & z_2^{(2)} & \cdots & z_2^{(m)} \\
\vdots & \vdots &  \ddots &  \\
z_c^{(1)} & z_c^{(2)} &   & z_c^{(m)} \\
\end{bmatrix} \
= \
\begin{bmatrix} \
w_{1,1} & w_{1,2} & \cdots & w_{1,D} \\
w_{2,1} & w_{2,2} & \cdots & w_{2,D} \\
\vdots & \vdots & \ddots &  \\
w_{c,1} & w_{c,2} &  & w_{c,D} \\
\end{bmatrix} \
\
\begin{bmatrix} \
x_1^{(1)} & x_1^{(2)} & \cdots & x_1^{(m)} \\
x_2^{(1)} & x_2^{(2)} & \cdots & x_2^{(m)} \\
\vdots & \vdots &  \ddots &  \\
x_D^{(1)} & x_D^{(2)} &   & x_D^{(m)} \\
\end{bmatrix}
 + \
\begin{bmatrix} \
b_1 \\
b_2 \\
\vdots\\
b_c \\
\end{bmatrix} $

$ \mathbf{\hat{y}} = ({1 + e^{-\mathbf{z}}})^{-1} $

In [ ]:
# Initialize our weights and biases to zero
w = np.zeros((D,c))
b = np.zeros((c,1))

In [ ]:
z = lambda w,X,b: np.dot(w.T, X) + b
yHat = lambda z: 1 / (1 + np.exp(-z))

$ \mathcal{L}( \mathbf{y}^{(i)}, \mathbf{\hat{y}}^{(i)}) = -(\
\mathbf{y}^{(i)} \cdot \ln{\mathbf{\hat{y}}^{(i)}} + \
(1 - \mathbf{y}^{(i)}) \cdot \ln{(1 - \mathbf{\hat{y}}^{(i)})})
$

$
\mathbf{J} = -\frac{1}{m} \sum_i^m \mathcal{L}( \mathbf{y}^{(i)}, \mathbf{\hat{y}}^{(i)})
$

In [ ]:
J = lambda y, yHat: -np.mean(y * np.log(yHat) + (1 - y) * np.log(1 - yHat))

$$ \frac{\delta \mathbf{J}}{\delta \mathbf{w}} = \
\frac{\delta \mathbf{J}}{\delta \mathbf{\hat{y}}} \cdot \
\frac{\delta \mathbf{\hat{y}}}{\delta \mathbf{z}} \cdot \
\frac{\delta \mathbf{z}}{\delta \mathbf{w}}
$$

$$ \frac{\delta \mathbf{J}}{\delta \mathbf{\hat{y}}} = \
\frac{-\mathbf{y}}{\mathbf{\hat{y}}} + \frac{1 - \mathbf{y}}{1 - \mathbf{\hat{y}}}
$$

$$ \frac{\delta \mathbf{\hat{y}}}{\delta \mathbf{z}} = \mathbf{\hat{y}}(1 - \mathbf{\hat{y}})$$

$$ \frac{\delta \mathbf{\hat{z}}}{\delta \mathbf{w}} = \mathbf{X} $$

$$ \frac{\delta \mathbf{J}}{\delta \mathbf{w}} = \
\frac{\delta \mathbf{J}}{\delta \mathbf{\hat{y}}} \cdot \
\frac{\delta \mathbf{\hat{y}}}{\delta \mathbf{z}} \cdot \
\frac{\delta \mathbf{z}}{\delta \mathbf{w}} = \
\Big[\frac{-\mathbf{y}}{\mathbf{\hat{y}}} + \frac{1 - \mathbf{y}}{1 - \mathbf{\hat{y}}}\Big] \dot \
\mathbf{\hat{y}}(1 - \mathbf{\hat{y}}) \cdot \mathbf{X} = \
(\mathbf{\hat{y}} - \mathbf{y}) \cdot \mathbf{X}
$$

In [ ]:
dJdw = lambda X, y, yHat: np.dot(X, (yHat - y).T) / len(X)
dJdb = lambda y, yHat: np.mean(yHat - y, axis=1, keepdims=True)

In [ ]:
a = yHat(z(w,train_x,b))
assert(a.shape == train_y.shape)

dw = dJdw(train_x, train_y, a)
assert(dw.shape == w.shape)

db = dJdb(train_y, a)
assert(db.shape == b.shape)

In [ ]:
epochs = 100000
alpha = 0.001

def descend(X, y, w, b):
    
    for i in range(epochs):
        a = yHat(z(w,X,b))
        
        if (i % (epochs / 10) == 0): print("cost at iteration {}: {}".format(i,J(y, a)))
            
        w = w - alpha * dJdw(X, y, a)
        b = b - alpha * dJdb(y, a)
    
    return w, b

In [ ]:
c, D, m = dimensions(train_x, train_y)

w = np.zeros((D,c))
b = np.zeros((c,1))

w, b = descend(train_x, train_y, w, b)

In [ ]:
accuracy = lambda y, yhat: np.mean(y.argmax(axis=0) == yhat.argmax(axis=0)) * 100

print("Accuracy on train set: {} %".format(accuracy(train_y, yHat(z(w,train_x,b)))))
print("Accuracy on test set: {}%".format(accuracy(test_y, yHat(z(w,test_x,b)))))

In [ ]:
for sample in range(20):
    actual = test_y.argmax(axis=0)[sample]
    prediction = yHat(z(w,test_x,b)).argmax(axis=0)[sample] 
    print(actual, prediction, actual == prediction)